In [2]:
import pandas as pd, numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [3]:
# NB: this requires openpyxl to be installed
xl = pd.ExcelFile("/data/Dataset.xlsx")
# extract sheets that start with consumer
consumer_sheets = [name for name in xl.sheet_names
                   if name.startswith("Consumer")]
# TODO: take all consumer sheets, not only the first 10 (= delete next line)
#consumer_sheets = consumer_sheets[:20]

### Creation of dataset for task 1

In [4]:
# mapping consumer to DataFrame containing tabular data
cons2df = {name: xl.parse(name).drop(columns=["Unnamed: 0","Total Consumption"])
           for name in tqdm(consumer_sheets)}

100%|██████████| 50/50 [04:18<00:00,  5.16s/it]


In [6]:
# sum over all columns that are not Periods (which is at index 0)
sum_cols = cons2df["Consumer1"].columns[1:].tolist()
print(sum_cols)
for name, df in cons2df.items():
    df["Total Consumption"] = df[sum_cols].sum(axis=1)

['AC', 'Dish washer', 'Washing Machine', 'Dryer', 'Water heater', 'TV', 'Microwave', 'Kettle', 'Lighting', 'Refrigerator', 'Total Consumption']


,Periods,AC,Dish washer,Washing Machine,Dryer,Water heater,TV,Microwave,Kettle,Lighting,Refrigerator,Total Consumption
0,1,0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,2,0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,3,0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,4,0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,5,0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
35131,92,0,0.0,0.0,0.0,0.346,0.046792,0.0,0.0,0.219333,0.0,1.224251
35132,93,0,0.0,0.0,0.0,0.207,0.047054,0.0,0.0,0.217667,0.0,0.943441
35133,94,0,0.0,0.0,0.0,0.373,0.046454,0.0,0.0,0.218667,0.0,1.276241
35134,95,0,0.0,0.0,0.0,0.079,0.000000,0.0,0.0,0.063333,0.0,0.284667


In [25]:
# constants based on 15min intervals
day_length = 4 * 24
week_window = day_length * 7

# iterate over DataFrames and extract x = week -> y = next day pairs
xs, ys = [], []
for df in cons2df.values():
  # extract the total consumption data from the dataset
  data = df["Total Consumption"].values

  # obtain start and end values of the window
  # NB: the window end includes the day to be predicted
  starts = np.arange(0, len(data) - week_window - day_length + 1, day_length)
  ends = starts + week_window + day_length # end includes the day to predict
  cons_xs, cons_ys = [], [] # save data for this consumer
  for start, end in zip(starts, ends):
    # extract the 8-day window from the dataset
    window = data[start:end]
    # split the window in week data and the day to predict
    x, y = window[:-day_length], window[-day_length:]
    # put the samples in the dataset
    x_days = 0
    # get the consumption per day
    week = []
    for i in range(0, len(x), day_length):
      values = x[i:i+day_length]
      week.append(sum(values))
    cons_xs.append(week)
    cons_ys.append(sum(y))
  assert end == len(data) # ensure that we covered all of the input data
  xs.append(np.stack(cons_xs))
  ys.append(np.stack(cons_ys))
# create length dimension
xs, ys = np.stack(xs), np.stack(ys)
# swap day (=N) and household dimensions such that we can sample days for
# dataset creation
xs = xs.transpose(1, 0, 2)
ys = ys.transpose(1, 0)
xs.shape, ys.shape

((359, 50, 7), (359, 50))

In [26]:
# generate train/validation/test splits with 80/10/10 ratio
xtr, xvalte, ytr, yvalte = train_test_split(xs, ys, test_size=.2, shuffle=False)
xval, xte, yval, yte = train_test_split(xvalte, yvalte, test_size=.5, shuffle=False)
xtr = xtr.transpose(1, 0, 2)
ytr = ytr.transpose(1, 0)
xval = xval.transpose(1, 0, 2)
yval = yval.transpose(1, 0)
xte = xte.transpose(1, 0, 2)
yte = yte.transpose(1, 0)
xs = xs.transpose(1, 0, 2)
ys = ys.transpose(1, 0)

In [27]:
np.savez("/data/task1_train", x=xtr, y=ytr)
np.savez("/data/task1_val", x=xval, y=yval)
np.savez("/data/task1_test", x=xte, y=yte)
np.savez("/data/task1_data", x=xs, y=ys)

### Creation of dataset for task 2

In [9]:
cons2df = {name: xl.parse(name).drop(columns=["Unnamed: 0","Periods","Total Consumption"])
           for name in tqdm(consumer_sheets)}

100%|██████████| 50/50 [04:26<00:00,  5.32s/it]


In [19]:
# constants based on 15min intervals
day_length = 4 * 24
mapping = {'AC': 0, 'Dish washer': 1, 'Washing Machine': 2, 'Dryer': 3, 'Water heater': 4, 'TV': 5, 'Microwave': 6, 'Kettle': 7, 'Lighting': 8, 'Refrigerator': 9}

# iterate over DataFrames and extract x = week -> y = next day pairs
xs, ys = [], []
for df in cons2df.values():
    cons_xs = []
    cons_ys = []
    for (appliance, data) in df.items():
        day_consumption = []
        count = 0
        for i in range(len(data)):
            if count == day_length:
                cons_xs.append(day_consumption)
                cons_ys.append(mapping[appliance])
                day_consumption = []
                count = 0
            day_consumption.append(data[i])
            count += 1
    xs.append(np.stack(cons_xs))
    ys.append(np.stack(cons_ys))
# create length dimension
xs, ys = np.stack(xs), np.stack(ys)
# swap day (=N) and household dimensions such that we can sample days for
# dataset creation
xs = xs.transpose(1, 0, 2)
ys = ys.transpose(1, 0)
xs.shape, ys.shape
xs[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [23]:
# generate train/validation/test splits with 80/10/10 ratio
xtr, xvalte, ytr, yvalte = train_test_split(xs, ys, test_size=.2, shuffle=True)
xval, xte, yval, yte = train_test_split(xvalte, yvalte, test_size=.5, shuffle=True)
xtr = xtr.transpose(1, 0, 2)
ytr = ytr.transpose(1, 0)
xval = xval.transpose(1, 0, 2)
yval = yval.transpose(1, 0)
xte = xte.transpose(1, 0, 2)
yte = yte.transpose(1, 0)
xs = xs.transpose(1, 0, 2)
ys = ys.transpose(1, 0)

In [24]:
np.savez("/data/task2_train", x=xtr, y=ytr)
np.savez("/data/task2_val", x=xval, y=yval)
np.savez("/data/task2_test", x=xte, y=yte)
np.savez("/data/task2_data", x=xs, y=ys)